# IMDB: Xception

In [12]:
import keras
import os, shutil
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
%matplotlib inline

train_dir = 'data\\imdb\\train'
test_dir = 'data\\imdb\\test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32
)

Found 221477 images belonging to 62 classes.
Found 15843 images belonging to 62 classes.


# Defining custom metrix

In [4]:
def five_claas_diff(y_true, y_pred):
    return K.cast(K.abs(K.argmax(y_true, axis=-1) - K.argmax(y_pred, axis=-1)) <= 5,
                  K.floatx())

def ten_claas_diff(y_true, y_pred):
    return K.cast(K.abs(K.argmax(y_true, axis=-1) - K.argmax(y_pred, axis=-1)) <= 10,
                  K.floatx())


In [5]:
from keras.applications.xception import Xception

conv_base = Xception(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))
                 
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [8]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(62, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               26214912  
_________________________________________________________________
dense_4 (Dense)              (None, 62)                31806     
Total params: 47,108,198
Trainable params: 26,246,718
Non-trainable params: 20,861,480
_________________________________________________________________


In [9]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy, five_claas_diff, ten_claas_diff]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=2275, 
                              epochs=2, 
                              validation_data=test_generator,
                              callbacks=[keras.callbacks.ModelCheckpoint('checkpoints\\imdb_xception.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)],
                              validation_steps=250)

This is the number of trainable weights before freezing the conv base: 4
Epoch 1/2
2275/2275 [==============================] - 605s 266ms/step - loss: 3.8871 - categorical_accuracy: 0.0368 - five_claas_diff: 0.3641 - ten_claas_diff: 0.6249 - val_loss: 4.1664 - val_categorical_accuracy: 0.0335 - val_five_claas_diff: 0.3712 - val_ten_claas_diff: 0.6445
Epoch 2/2
2275/2275 [==============================] - 464s 204ms/step - loss: 3.8078 - categorical_accuracy: 0.0422 - five_claas_diff: 0.3809 - ten_claas_diff: 0.6454 - val_loss: 4.8927 - val_categorical_accuracy: 0.0355 - val_five_claas_diff: 0.3618 - val_ten_claas_diff: 0.6206


In [13]:
from keras.models import load_model
try:
    model = load_model('checkpoints\\imdb_xception.h5', custom_objects={'five_claas_diff': five_claas_diff, 'ten_claas_diff':ten_claas_diff})
except:
    pass

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:304: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


# Fine tuning

In [14]:
conv_base.trainable = True
        
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy, five_claas_diff, ten_claas_diff]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=2275, 
                              epochs=30, 
                              validation_data=test_generator, 
                              callbacks=[
                                  keras.callbacks.ModelCheckpoint('checkpoints\\imdb_xception.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1),
                                  keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
                              ],
                              validation_steps=250)

Epoch 1/30
2275/2275 [==============================] - 746s 328ms/step - loss: 3.7066 - categorical_accuracy: 0.0470 - five_claas_diff: 0.4369 - ten_claas_diff: 0.7056 - val_loss: 3.7514 - val_categorical_accuracy: 0.0471 - val_five_claas_diff: 0.4519 - val_ten_claas_diff: 0.7384
Epoch 2/30
2275/2275 [==============================] - 725s 319ms/step - loss: 3.6082 - categorical_accuracy: 0.0527 - five_claas_diff: 0.4868 - ten_claas_diff: 0.7544 - val_loss: 3.6826 - val_categorical_accuracy: 0.0545 - val_five_claas_diff: 0.5076 - val_ten_claas_diff: 0.7731
Epoch 3/30
2275/2275 [==============================] - 724s 318ms/step - loss: 3.5712 - categorical_accuracy: 0.0590 - five_claas_diff: 0.5120 - ten_claas_diff: 0.7719 - val_loss: 3.6154 - val_categorical_accuracy: 0.0605 - val_five_claas_diff: 0.5212 - val_ten_claas_diff: 0.7737
Epoch 4/30
2275/2275 [==============================] - 724s 318ms/step - loss: 3.4822 - categorical_accuracy: 0.0668 - five_claas_diff: 0.5574 - ten_claa

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
val_5_acc = history.history['val_five_claas_diff']
val_10_acc = history.history['val_ten_claas_diff']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.plot(epochs, val_5_acc, 'r', label='5 class val acc')
plt.plot(epochs, val_10_acc, 'g', label='10 class val acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()